In [6]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc


In [7]:
# 가을14, 15년도 데이터 전체에 고객마다 라벨값을 추가한 데이터 read
cust_df = pd.read_pickle('dataset/full_data_with_label.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/full_data_with_label.pkl'

### 생각할 점
- 모든 데이터를 다 쓸 것인가

In [13]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5817736 entries, 0 to 3436519
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   RECEIPTNO    object        
 1   CUSTNO       object        
 2   PURCHDATE    datetime64[ns]
 3   PURCHTIME    int64         
 4   AFFIL        object        
 5   CLASSCODEL   int32         
 6   CLASSCODEM   object        
 7   CLASSCODES   object        
 8   STORENO      object        
 9   GENDER       object        
 10  AGEGROUP     object        
 11  RESIDENCE    object        
 12  PURCHAMOUNT  int64         
 13  Year         int64         
 14  AFFIL_CLASS  object        
 15  spent_less   int64         
dtypes: datetime64[ns](1), int32(1), int64(4), object(10)
memory usage: 732.4+ MB


In [21]:
# 아직 원핫인코딩을 하지 않았기 때문에 바로 모델을 돌릴 수 있는 변수 purchtime, purchamount만 남김
ddf = cust_df[['PURCHTIME','PURCHAMOUNT','spent_less']]
y_labels = ddf['spent_less']
X_features= ddf.drop('spent_less', axis=1)

In [22]:
ddf.spent_less.value_counts()

0    5489490
1     328246
Name: spent_less, dtype: int64

In [23]:
# 비율확인
print(ddf['spent_less'].value_counts())
unsatisfied_cnt = ddf[cust_df['spent_less'] == 1].spent_less.count()
total_cnt = ddf.spent_less.count()
print('spent_less 비율은 {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

0    5489490
1     328246
Name: spent_less, dtype: int64
spent_less 비율은 0.06


- 6%가 감소.

In [24]:
# 학습, 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0) # stratify=y_labels?
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 Shape:(4654188, 2), 테스트 세트 Shape:(1163548, 2)
 학습 세트 레이블 값 분포 비율
0    0.943588
1    0.056412
Name: spent_less, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.94354
1    0.05646
Name: spent_less, dtype: float64


In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [30]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=100)
evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="auc", eval_set=evals,
                verbose=True)
lgbm_pred = lgbm_clf.predict(X_test) # 예측
lgbm_pred_proba = lgbm_clf.predict_proba(X_test)[:, 1]

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.569355	valid_0's binary_logloss: 0.216796
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.569713	valid_0's binary_logloss: 0.216537
[3]	valid_0's auc: 0.569845	valid_0's binary_logloss: 0.216323
[4]	valid_0's auc: 0.569844	valid_0's binary_logloss: 0.216148
[5]	valid_0's auc: 0.569763	valid_0's binary_logloss: 0.216003
[6]	valid_0's auc: 0.570222	valid_0's binary_logloss: 0.215884
[7]	valid_0's auc: 0.570388	valid_0's binary_logloss: 0.215783
[8]	valid_0's auc: 0.570548	valid_0's binary_logloss: 0.215699
[9]	valid_0's auc: 0.570564	valid_0's binary_logloss: 0.215631
[10]	valid_0's auc: 0.570783	valid_0's binary_logloss: 0.215572
[11]	valid_0's auc: 0.570869	valid_0's binary_logloss: 0.215523
[12]	valid_0's auc: 0.570918	valid_0's binary_logloss: 0.215483
[13]	valid_0's auc: 0.571012	valid_0's binary_logloss: 0.215449
[14]	valid_0's auc: 0.571133	valid_0's binary_logloss: 0.21542
[15]	valid_0's auc: 0.571246	valid_0's binary_logloss

In [ ]:
from sklearn.model_selection import GridSearchCV

LGBM_clf = LGBMClassifier(n_estimators=100)

params = {'num_leaves': [32,64],
         'max_depth':[128,160],
         'min_child_samples':[60,100],
         'subsample':[0.8,1]}
gridcv = GridSearchCV(LGBM_clf,param_grid=params)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc', \
                      eval_set=[(X_train,y_train),(X_test,y_test)])
print('GCV 최적 파라미터:', gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('AUC:{0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.569487	valid_0's binary_logloss: 0.216663	valid_1's auc: 0.568492	valid_1's binary_logloss: 0.216802
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.570133	valid_0's binary_logloss: 0.216402	valid_1's auc: 0.569381	valid_1's binary_logloss: 0.216543
[3]	valid_0's auc: 0.570188	valid_0's binary_logloss: 0.216188	valid_1's auc: 0.569521	valid_1's binary_logloss: 0.216331
[4]	valid_0's auc: 0.570299	valid_0's binary_logloss: 0.216011	valid_1's auc: 0.569689	valid_1's binary_logloss: 0.216155
[5]	valid_0's auc: 0.570547	valid_0's binary_logloss: 0.215864	valid_1's auc: 0.569875	valid_1's binary_logloss: 0.21601
[6]	valid_0's auc: 0.570887	valid_0's binary_logloss: 0.215742	valid_1's auc: 0.570209	valid_1's binary_logloss: 0.215889
[7]	valid_0's auc: 0.571161	valid_0's binary_logloss: 0.215641	valid_1's auc: 0.570448	valid_1's binary_logloss: 0.21579
[8]	valid_0's auc: 0.571272	valid_0's binary_logloss: 0.215556	valid_1's auc: 0.570585	

[68]	valid_0's auc: 0.575079	valid_0's binary_logloss: 0.214976	valid_1's auc: 0.572935	valid_1's binary_logloss: 0.215212
[69]	valid_0's auc: 0.575157	valid_0's binary_logloss: 0.214973	valid_1's auc: 0.573011	valid_1's binary_logloss: 0.215209
[70]	valid_0's auc: 0.57516	valid_0's binary_logloss: 0.214972	valid_1's auc: 0.573013	valid_1's binary_logloss: 0.215209
[71]	valid_0's auc: 0.57517	valid_0's binary_logloss: 0.214971	valid_1's auc: 0.573018	valid_1's binary_logloss: 0.215209
[72]	valid_0's auc: 0.575239	valid_0's binary_logloss: 0.214968	valid_1's auc: 0.573044	valid_1's binary_logloss: 0.215208
[73]	valid_0's auc: 0.575276	valid_0's binary_logloss: 0.214966	valid_1's auc: 0.573041	valid_1's binary_logloss: 0.215208
[74]	valid_0's auc: 0.5753	valid_0's binary_logloss: 0.214965	valid_1's auc: 0.573059	valid_1's binary_logloss: 0.215207
[75]	valid_0's auc: 0.575313	valid_0's binary_logloss: 0.214964	valid_1's auc: 0.573057	valid_1's binary_logloss: 0.215207
[76]	valid_0's auc: 

[135]	valid_0's auc: 0.576618	valid_0's binary_logloss: 0.214893	valid_1's auc: 0.573381	valid_1's binary_logloss: 0.215192
[136]	valid_0's auc: 0.576631	valid_0's binary_logloss: 0.214892	valid_1's auc: 0.573375	valid_1's binary_logloss: 0.215192
[137]	valid_0's auc: 0.576658	valid_0's binary_logloss: 0.214891	valid_1's auc: 0.573384	valid_1's binary_logloss: 0.215191
[138]	valid_0's auc: 0.576686	valid_0's binary_logloss: 0.214889	valid_1's auc: 0.573395	valid_1's binary_logloss: 0.215191
[139]	valid_0's auc: 0.576697	valid_0's binary_logloss: 0.214889	valid_1's auc: 0.573399	valid_1's binary_logloss: 0.215191
[140]	valid_0's auc: 0.57672	valid_0's binary_logloss: 0.214887	valid_1's auc: 0.573405	valid_1's binary_logloss: 0.215191
[141]	valid_0's auc: 0.576748	valid_0's binary_logloss: 0.214886	valid_1's auc: 0.573415	valid_1's binary_logloss: 0.21519
[142]	valid_0's auc: 0.576773	valid_0's binary_logloss: 0.214885	valid_1's auc: 0.573412	valid_1's binary_logloss: 0.21519
[143]	valid

[1]	valid_0's auc: 0.569836	valid_0's binary_logloss: 0.216658	valid_1's auc: 0.569251	valid_1's binary_logloss: 0.216795
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.570167	valid_0's binary_logloss: 0.216396	valid_1's auc: 0.569219	valid_1's binary_logloss: 0.216536
[3]	valid_0's auc: 0.570505	valid_0's binary_logloss: 0.21618	valid_1's auc: 0.56964	valid_1's binary_logloss: 0.216324
[4]	valid_0's auc: 0.57064	valid_0's binary_logloss: 0.216001	valid_1's auc: 0.569751	valid_1's binary_logloss: 0.216149
[5]	valid_0's auc: 0.570662	valid_0's binary_logloss: 0.215854	valid_1's auc: 0.569716	valid_1's binary_logloss: 0.216005
[6]	valid_0's auc: 0.571039	valid_0's binary_logloss: 0.215732	valid_1's auc: 0.570193	valid_1's binary_logloss: 0.215884
[7]	valid_0's auc: 0.571216	valid_0's binary_logloss: 0.21563	valid_1's auc: 0.570334	valid_1's binary_logloss: 0.215785
[8]	valid_0's auc: 0.571286	valid_0's binary_logloss: 0.215545	valid_1's auc: 0.57035	val

[68]	valid_0's auc: 0.575167	valid_0's binary_logloss: 0.214961	valid_1's auc: 0.572804	valid_1's binary_logloss: 0.215204
[69]	valid_0's auc: 0.575181	valid_0's binary_logloss: 0.214959	valid_1's auc: 0.572803	valid_1's binary_logloss: 0.215204
[70]	valid_0's auc: 0.575211	valid_0's binary_logloss: 0.214957	valid_1's auc: 0.572818	valid_1's binary_logloss: 0.215203
[71]	valid_0's auc: 0.575225	valid_0's binary_logloss: 0.214956	valid_1's auc: 0.572817	valid_1's binary_logloss: 0.215204
[72]	valid_0's auc: 0.575249	valid_0's binary_logloss: 0.214955	valid_1's auc: 0.572809	valid_1's binary_logloss: 0.215203
[73]	valid_0's auc: 0.575251	valid_0's binary_logloss: 0.214954	valid_1's auc: 0.57281	valid_1's binary_logloss: 0.215203
[74]	valid_0's auc: 0.575307	valid_0's binary_logloss: 0.214952	valid_1's auc: 0.572842	valid_1's binary_logloss: 0.215202
[75]	valid_0's auc: 0.575341	valid_0's binary_logloss: 0.21495	valid_1's auc: 0.572833	valid_1's binary_logloss: 0.215203
[76]	valid_0's auc

...

[184]	valid_0's auc: 0.577581	valid_0's binary_logloss: 0.214834	valid_1's auc: 0.573698	valid_1's binary_logloss: 0.215172
[185]	valid_0's auc: 0.577589	valid_0's binary_logloss: 0.214834	valid_1's auc: 0.573696	valid_1's binary_logloss: 0.215172
[186]	valid_0's auc: 0.577606	valid_0's binary_logloss: 0.214833	valid_1's auc: 0.573695	valid_1's binary_logloss: 0.215172
[187]	valid_0's auc: 0.577642	valid_0's binary_logloss: 0.214831	valid_1's auc: 0.573709	valid_1's binary_logloss: 0.215171
[188]	valid_0's auc: 0.577668	valid_0's binary_logloss: 0.21483	valid_1's auc: 0.573717	valid_1's binary_logloss: 0.215171
[189]	valid_0's auc: 0.57768	valid_0's binary_logloss: 0.21483	valid_1's auc: 0.573716	valid_1's binary_logloss: 0.215171
[190]	valid_0's auc: 0.577687	valid_0's binary_logloss: 0.214829	valid_1's auc: 0.57372	valid_1's binary_logloss: 0.215171
[191]	valid_0's auc: 0.577691	valid_0's binary_logloss: 0.214829	valid_1's auc: 0.573715	valid_1's binary_logloss: 0.215171
[192]	valid_

[49]	valid_0's auc: 0.574545	valid_0's binary_logloss: 0.215002	valid_1's auc: 0.572938	valid_1's binary_logloss: 0.215203
[50]	valid_0's auc: 0.574591	valid_0's binary_logloss: 0.214999	valid_1's auc: 0.572963	valid_1's binary_logloss: 0.215202
[51]	valid_0's auc: 0.57462	valid_0's binary_logloss: 0.214997	valid_1's auc: 0.572961	valid_1's binary_logloss: 0.215202
[52]	valid_0's auc: 0.574629	valid_0's binary_logloss: 0.214996	valid_1's auc: 0.572958	valid_1's binary_logloss: 0.215201
[53]	valid_0's auc: 0.574697	valid_0's binary_logloss: 0.214994	valid_1's auc: 0.572999	valid_1's binary_logloss: 0.2152
[54]	valid_0's auc: 0.574709	valid_0's binary_logloss: 0.214992	valid_1's auc: 0.573001	valid_1's binary_logloss: 0.215199
[55]	valid_0's auc: 0.574737	valid_0's binary_logloss: 0.21499	valid_1's auc: 0.573	valid_1's binary_logloss: 0.215198
[56]	valid_0's auc: 0.57476	valid_0's binary_logloss: 0.214988	valid_1's auc: 0.572989	valid_1's binary_logloss: 0.215198
[57]	valid_0's auc: 0.57

[116]	valid_0's auc: 0.576321	valid_0's binary_logloss: 0.214904	valid_1's auc: 0.57351	valid_1's binary_logloss: 0.215174
[117]	valid_0's auc: 0.576331	valid_0's binary_logloss: 0.214903	valid_1's auc: 0.573513	valid_1's binary_logloss: 0.215174
[118]	valid_0's auc: 0.576338	valid_0's binary_logloss: 0.214903	valid_1's auc: 0.573518	valid_1's binary_logloss: 0.215174
[119]	valid_0's auc: 0.576364	valid_0's binary_logloss: 0.214902	valid_1's auc: 0.573517	valid_1's binary_logloss: 0.215174
[120]	valid_0's auc: 0.576372	valid_0's binary_logloss: 0.214901	valid_1's auc: 0.573515	valid_1's binary_logloss: 0.215174
[121]	valid_0's auc: 0.576399	valid_0's binary_logloss: 0.2149	valid_1's auc: 0.573527	valid_1's binary_logloss: 0.215174
[122]	valid_0's auc: 0.576403	valid_0's binary_logloss: 0.214899	valid_1's auc: 0.573527	valid_1's binary_logloss: 0.215174
[123]	valid_0's auc: 0.576436	valid_0's binary_logloss: 0.214898	valid_1's auc: 0.573534	valid_1's binary_logloss: 0.215174
[124]	valid

[12]	valid_0's auc: 0.57197	valid_0's binary_logloss: 0.21532	valid_1's auc: 0.571158	valid_1's binary_logloss: 0.215476
[13]	valid_0's auc: 0.572102	valid_0's binary_logloss: 0.215283	valid_1's auc: 0.571245	valid_1's binary_logloss: 0.215441
[14]	valid_0's auc: 0.572234	valid_0's binary_logloss: 0.215252	valid_1's auc: 0.571397	valid_1's binary_logloss: 0.215411
[15]	valid_0's auc: 0.572333	valid_0's binary_logloss: 0.215227	valid_1's auc: 0.571436	valid_1's binary_logloss: 0.215388
[16]	valid_0's auc: 0.572432	valid_0's binary_logloss: 0.215203	valid_1's auc: 0.571581	valid_1's binary_logloss: 0.215364
[17]	valid_0's auc: 0.572514	valid_0's binary_logloss: 0.215184	valid_1's auc: 0.57165	valid_1's binary_logloss: 0.215347
[18]	valid_0's auc: 0.572608	valid_0's binary_logloss: 0.215167	valid_1's auc: 0.571767	valid_1's binary_logloss: 0.215331
[19]	valid_0's auc: 0.572687	valid_0's binary_logloss: 0.215152	valid_1's auc: 0.571824	valid_1's binary_logloss: 0.215316
[20]	valid_0's auc:

[79]	valid_0's auc: 0.575427	valid_0's binary_logloss: 0.214948	valid_1's auc: 0.573403	valid_1's binary_logloss: 0.215177
[80]	valid_0's auc: 0.575515	valid_0's binary_logloss: 0.214945	valid_1's auc: 0.57347	valid_1's binary_logloss: 0.215175
[81]	valid_0's auc: 0.575533	valid_0's binary_logloss: 0.214944	valid_1's auc: 0.573463	valid_1's binary_logloss: 0.215175
[82]	valid_0's auc: 0.57554	valid_0's binary_logloss: 0.214943	valid_1's auc: 0.573465	valid_1's binary_logloss: 0.215175
[83]	valid_0's auc: 0.575564	valid_0's binary_logloss: 0.214942	valid_1's auc: 0.573459	valid_1's binary_logloss: 0.215175
[84]	valid_0's auc: 0.575566	valid_0's binary_logloss: 0.214941	valid_1's auc: 0.573461	valid_1's binary_logloss: 0.215175
[85]	valid_0's auc: 0.575579	valid_0's binary_logloss: 0.21494	valid_1's auc: 0.573471	valid_1's binary_logloss: 0.215174
[86]	valid_0's auc: 0.575623	valid_0's binary_logloss: 0.214938	valid_1's auc: 0.573522	valid_1's binary_logloss: 0.215173
[87]	valid_0's auc: